## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

### Задание 1.

#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [2]:
import requests

import time

import pandas as pd

import datetime

In [2]:
KEYWORDS = ['python', 'парсинг', 'Athelas']

In [4]:
# в переменной res получаем HTML код страницы из переменной URL

URL = 'https://habr.com/ru/all/'
res = requests.get(URL)
res

<Response [200]>

In [3]:
from bs4 import BeautifulSoup

In [5]:
#  получаем текст HTML кода из переменной res и упорядочиваем этот текст
soup = BeautifulSoup(res.text, 'html.parser')


In [6]:
# получаем (из переменной soup) список (list) частей текста HTML кода, содержащую информацию о каждом посте

posts = soup.find_all('article', class_='post')
#posts

In [7]:
len(posts)

20

In [28]:
?map
# a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))

post_class_link = list(map(lambda x: x.find('h2', class_='post__title'), posts))

In [42]:
# из каждого элемента списка (переменная posts), получаем часть HTML кода, содержащую link на полный текст статьи

#post_class_link = posts[0].find('h2', class_='post__title')
#print(post_class_link)

In [44]:
# из переменной post_class_link извлекаем ссылку на полный текст статьи (из HTML кода) 
post_link = post_class_link.find('a').get('href')


post_link_list = list(map(lambda x: x.find('a').get('href'), post_class_link))
#print(post_link_list)

In [47]:
# из переменной post_class_link получаем заголовок статьи, как текст
#post_class_link.text

title_list = list(map(lambda x: x.text, post_class_link))
title_list

title_post = post_class_link[0].text
title_post

'\nГде работать в ИТ в 2020: OUTSIDE\n'

In [1]:
# получаем часть HTML кода по ссылке (переменная post_link)
soup_for_full_text = BeautifulSoup(requests.get(post_link).text, 'html.parser')
# soup_for_full

NameError: name 'BeautifulSoup' is not defined

In [40]:
# по ссылке получаем полный текст статьи
text = soup_for_full_text.find('div', class_='post__text').text
#text

In [13]:
KEYWORDS[1] in text

False

In [41]:
# получаем текст превью каждого поста из переменной posts

text_preview = posts[0].find('div', class_='post__text').text
text_preview

'\nСегодня расскажем (и даже покажем), как обстоят дела с внутренними процессами и корпоративной жизнью в digital-агентстве OUTSIDE, ранее известном как Kodix. На наши вопросы о найме, условиях работы, команде и технологиях ответили: гендиректор Иван Пипченко, ведущий эйчар Наталия Лебедева и техлид Антон Бармин.\n'

In [15]:
text_date = posts[0].find('span', class_='post__time').text
text_date 

'сегодня в 12:57'

In [17]:
def change_str_habr_date(str_date):
    import datetime
    
    months = {'1':'января', '2':'февраля','3':'марта','4':'апреля','5':'мая','6':'июня','7':'июля','8':'августа','9':'сентября','10':'октября','11':'ноября','12':'декабря'}
    date_1 = 'сегодня'
    date_2 = 'вчера'
    
    if date_1 in str_date:
        time = datetime.datetime.strptime(str_date, 'сегодня в %H:%M').time()
        date = datetime.datetime.today().date()
        date_post = datetime.datetime.combine(date, time)
    elif date_2 in str_date:
        time = datetime.datetime.strptime(str_date, 'вчера в %H:%M').time()
        date = datetime.datetime.now().date() - datetime.timedelta(days=1) 
        date_post = datetime.datetime.combine(date, time)
    else:
        for key, values in months.items():
            if values in str_date:
                str_date = str_date.replace(values, key)
                date_post = datetime.datetime.strptime(str_date, '%d %m %Y в %H:%M')
    
    return date_post
                

In [19]:
change_str_habr_date(text_date)

datetime.datetime(2020, 12, 4, 12, 57)

In [53]:
URL = 'https://habr.com/ru/all/'
res = requests.get(URL)
soup = BeautifulSoup(res.text, 'html.parser')
posts = soup.find_all('article', class_='post')

row = dict()
habr_news = pd.DataFrame()

for post in posts:
    post_class_link = post.find('h2', class_='post__title')
    post_link =  post_class_link.find('a').get('href')
    title_post = post_class_link.text
    text_preview = post.find('div', class_='post__text').text
    text_date = post.find('span', class_='post__time').text
    date = change_str_habr_date(text_date)
    for word in KEYWORDS:
        if word in title_post or word in text_preview:
            row = {'date': date, 'link':post_link, 'title':title_post, 'preview':text_preview}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])

row

{}

#### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

### Задание 2.

#### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

#### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий GitHub. Не сможем проверить или помочь, если вы пришлете:

файлы;
архивы;
скриншоты кода.
Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?

Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:
1) Что я делаю?
2) Какого результата я ожидаю?
3) Как фактический результат отличается от ожидаемого?
4) Что я уже попробовал сделать, чтобы исправить проблему?

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.